<a href="https://colab.research.google.com/github/dtoralg/INESDI_Data-Science_ML_IA/blob/main/%5B02%5D%20-%20Modelos%20Supervisados%20Lineales/Supervisados_Lineales_Ejercicio_7_naive_bayes_diabetes_EXTENDIDO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Supervisados Lineales - Ejercicio 7 (extendido): naive_bayes_diabetes.ipynb

En este ejercicio abordaremos **tres variantes de Naive Bayes** aplicadas al dataset **Diabetes** de `scikit-learn`:
- **GaussianNB**: asume rasgos **continuos** con distribución normal condicionada a la clase.
- **CategoricalNB**: requiere rasgos **discretos/categóricos**; discretizaremos (binning) las variables continuas.
- **MultinomialNB**: diseñado para **conteos** (no negativos); reescalaremos a [0, 1] y discutiremos limitaciones.

Formularemos un problema de **clasificación binaria** a partir del objetivo continuo (progresión de la enfermedad), evaluaremos con **accuracy, ROC-AUC, matriz de confusión** y cerraremos con una **comparativa** clara de resultados.

### Objetivos
- **O1.** Entrenar y evaluar **GaussianNB**, **CategoricalNB** y **MultinomialNB**.
- **O2.** Comprender cómo las **suposiciones** de cada variante afectan el rendimiento.
- **O3.** Comparar resultados con métricas y **matrices de confusión**; razonar cuándo usar cada enfoque.

### Dataset
El dataset **Diabetes** contiene 442 pacientes y 10 predictores normalizados (edad, sexo, IMC, presión sanguínea y 6 medidas bioquímicas). El objetivo original es **continuo** (progresión al año). Para clasificación, lo **binarizaremos por la mediana**.

### 1. Importación de librerías y utilidades

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, CategoricalNB, MultinomialNB
from sklearn.preprocessing import KBinsDiscretizer, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, classification_report
sns.set(style='whitegrid')

def plot_confusion(cm, title, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['Real 0','Real 1'], ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Predicción')
    ax.set_ylabel('Real')
    return ax


### 2. Carga de datos y binarización del objetivo

In [ ]:
# Cargar dataset de Diabetes
data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y_cont = pd.Series(data.target, name='progression')

In [ ]:
# Binarizamos el objetivo: 1 si progresión > mediana, 0 en caso contrario
thr = y_cont.median()
y = (y_cont > thr).astype(int)
print('Umbral de binarización (mediana):', thr)
display(X.head())
X.shape, y.value_counts(normalize=True).round(3)

### 3. División en train/test (estratificada)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape, y_train.mean().round(3), y_test.mean().round(3)

### 4. Gaussian Naive Bayes (rasgos continuos)

**Idea:** Modela cada rasgo continuo condicionalmente como una Normal $\mathcal{N}(\mu, \sigma^2)$ por clase. Conviene cuando los rasgos son aproximadamente gaussianos o han sido estandarizados.

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

In [ ]:
y_pred_gnb = gnb.predict(X_test)
y_proba_gnb = gnb.predict_proba(X_test)[:, 1]

In [ ]:
acc_gnb = accuracy_score(y_test, y_pred_gnb)
auc_gnb = roc_auc_score(y_test, y_proba_gnb)
cm_gnb = confusion_matrix(y_test, y_pred_gnb)
print(f'GaussianNB -> Accuracy: {acc_gnb:.3f} | ROC-AUC: {auc_gnb:.3f}')
print(classification_report(y_test, y_pred_gnb, digits=3))
plot_confusion(cm_gnb, 'GaussianNB'); plt.show()

### 5. Categorical Naive Bayes (discretización de rasgos)

**Idea:** Convertimos rasgos continuos en **categorías** (bins). El modelo aprende $P(x_j = \text{categoría} \mid y)$ por cada rasgo. Se pierde granularidad, pero el ajuste a la suposición categórica puede mejorar según el caso.

In [ ]:
# Discretizamos en 5 bins equipoblados (quantile). encode='ordinal' produce categorías [0..n_bins-1]
kbd = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
X_train_disc = kbd.fit_transform(X_train)
X_test_disc  = kbd.transform(X_test)

In [ ]:
cnb = CategoricalNB()
cnb.fit(X_train_disc, y_train)

In [ ]:
y_pred_cnb = cnb.predict(X_test_disc)
y_proba_cnb = cnb.predict_proba(X_test_disc)[:, 1]

In [ ]:
acc_cnb = accuracy_score(y_test, y_pred_cnb)
auc_cnb = roc_auc_score(y_test, y_proba_cnb)
cm_cnb = confusion_matrix(y_test, y_pred_cnb)
print(f'CategoricalNB -> Accuracy: {acc_cnb:.3f} | ROC-AUC: {auc_cnb:.3f}')
print(classification_report(y_test, y_pred_cnb, digits=3))
plot_confusion(cm_cnb, 'CategoricalNB'); plt.show()

### 6. Multinomial Naive Bayes (rasgos no negativos / tipo conteo)

**Idea:** Pensado para **conteos** (p. ej., frecuencias de palabras). Requiere rasgos **no negativos**. Aquí haremos un **reescaleo Min–Max a [0, 1]**.

**Nota:** Los rasgos originales no son conteos; MultinomialNB puede rendir peor si la inductiva no encaja, pero sirve para mostrar el contraste entre variantes de NB.

In [ ]:
mms = MinMaxScaler()
X_train_mnb = mms.fit_transform(X_train)
X_test_mnb  = mms.transform(X_test)

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train_mnb, y_train)

In [ ]:
y_pred_mnb = mnb.predict(X_test_mnb)
y_proba_mnb = mnb.predict_proba(X_test_mnb)[:, 1]

In [ ]:
acc_mnb = accuracy_score(y_test, y_pred_mnb)
auc_mnb = roc_auc_score(y_test, y_proba_mnb)
cm_mnb = confusion_matrix(y_test, y_pred_mnb)
print(f'MultinomialNB -> Accuracy: {acc_mnb:.3f} | ROC-AUC: {auc_mnb:.3f}')
print(classification_report(y_test, y_pred_mnb, digits=3))
plot_confusion(cm_mnb, 'MultinomialNB'); plt.show()

### 7. Comparativa: matrices de confusión y métricas

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14,4))
plot_confusion(cm_gnb, f'GaussianNB\nAcc={acc_gnb:.2f}', ax=axes[0])
plot_confusion(cm_cnb, f'CategoricalNB\nAcc={acc_cnb:.2f}', ax=axes[1])
plot_confusion(cm_mnb, f'MultinomialNB\nAcc={acc_mnb:.2f}', ax=axes[2])
plt.tight_layout(); plt.show()

metrics = pd.DataFrame({
    'modelo': ['GaussianNB','CategoricalNB','MultinomialNB'],
    'accuracy': [acc_gnb, acc_cnb, acc_mnb],
    'roc_auc': [auc_gnb, auc_cnb, auc_mnb]
}).sort_values('accuracy', ascending=False)
metrics

### 8. Curvas ROC (opcional)

In [ ]:
fpr_g, tpr_g, _ = roc_curve(y_test, y_proba_gnb)
fpr_c, tpr_c, _ = roc_curve(y_test, y_proba_cnb)
fpr_m, tpr_m, _ = roc_curve(y_test, y_proba_mnb)

plt.figure(figsize=(6,5))
plt.plot(fpr_g, tpr_g, label=f'GaussianNB AUC={auc_gnb:.3f}')
plt.plot(fpr_c, tpr_c, label=f'CategoricalNB AUC={auc_cnb:.3f}')
plt.plot(fpr_m, tpr_m, label=f'MultinomialNB AUC={auc_mnb:.3f}')
plt.plot([0,1],[0,1],'k--', linewidth=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curvas ROC - Comparativa')
plt.legend(loc='lower right')
plt.show()

### Conclusiones
- **GaussianNB** suele ser una buena línea base cuando los rasgos son continuos y no presentan distribuciones muy patológicas.
- **CategoricalNB** puede mejorar si la discretización captura bien patrones por rango y reduce ruido/heteroscedasticidad.
- **MultinomialNB** es apropiado para **conteos**; en datos continuos reescalados, su inductiva no siempre encaja y puede rendir peor.

Al comparar **matrices de confusión**, presta atención a:
- **Falsos positivos** vs **falsos negativos** según lo que te preocupe minimizar.
- Cambios en **recall** y **precision** de la clase positiva al pasar de continuo→discreto→multinomial.

Elige el modelo no solo por **accuracy/AUC**, sino por la **distribución de errores** y las **suposiciones** que mejor se ajusten a tu caso.